In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb

In [3]:
df = pd.read_csv("/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/fall_window_dataset.csv")

x = df.drop(columns=['scene_id', 'label'])
y = df['label']

In [9]:
df.head()

,scene_id,label,sk_0_x_f0,sk_0_y_f0,sk_0_c_f0,sk_1_x_f0,sk_1_y_f0,sk_1_c_f0,sk_2_x_f0,sk_2_y_f0,...,bbox_dy_f8,bbox_cx_f9,bbox_cy_f9,bbox_w_f9,bbox_h_f9,bbox_dx_f9,bbox_dy_f9,vid_mean_flow,vid_max_flow,vid_mean_diff
0,02414_H_A_N_C1,0,561.924744,157.856964,0.951018,565.265686,154.273163,0.826885,558.665833,153.239014,...,-0.792201,468.180420,234.319560,75.809001,199.504883,0.127808,1.874090,0.114172,0.208973,67.363926
1,02414_H_A_N_C2,0,991.145264,223.081665,0.752328,999.516113,218.667999,0.809587,994.475525,214.358246,...,-1.548757,985.780924,252.187088,129.281576,270.704437,-3.379883,0.944219,0.224164,0.527882,65.907143
2,02414_H_A_N_C3,0,320.148224,176.466705,0.035564,317.452576,170.719543,0.005669,325.882874,167.741562,...,0.277659,481.064250,325.925975,132.959717,337.735453,0.594971,0.991043,0.185940,0.448204,62.061227
3,02414_H_A_N_C4,0,517.705750,171.797668,0.571712,515.639160,166.650391,0.106046,517.184387,167.068710,...,-0.660868,399.242676,278.633362,104.089030,262.730265,-3.509237,1.729406,0.120498,0.307896,54.137906
4,02414_H_A_N_C5,0,581.434692,237.248718,0.597601,579.601257,233.103058,0.147155,581.436401,232.045837,...,-1.096415,452.345418,330.782990,114.569661,299.084737,-4.480265,0.764516,0.150810,0.359749,55.246477


In [10]:
df.columns

Index(['scene_id', 'label', 'sk_0_x_f0', 'sk_0_y_f0', 'sk_0_c_f0', 'sk_1_x_f0',
       'sk_1_y_f0', 'sk_1_c_f0', 'sk_2_x_f0', 'sk_2_y_f0',
       ...
       'bbox_dy_f8', 'bbox_cx_f9', 'bbox_cy_f9', 'bbox_w_f9', 'bbox_h_f9',
       'bbox_dx_f9', 'bbox_dy_f9', 'vid_mean_flow', 'vid_max_flow',
       'vid_mean_diff'],
      dtype='object', length=575)

In [4]:
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.2, stratify=y, random_state=42
)

In [6]:
model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    n_jobs=-1
)

model.fit(
    x_train, y_train,
    eval_set=[(x_val, y_val)],
    verbose=50
)

[0]	validation_0-logloss:0.64300
[50]	validation_0-logloss:0.11992
[100]	validation_0-logloss:0.08083
[150]	validation_0-logloss:0.07197
[200]	validation_0-logloss:0.07015
[250]	validation_0-logloss:0.06956
[299]	validation_0-logloss:0.06883


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=-1,
              num_parallel_tree=None, ...)

In [7]:
y_pred = model.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       448
           1       0.98      0.97      0.97       343

    accuracy                           0.98       791
   macro avg       0.98      0.98      0.98       791
weighted avg       0.98      0.98      0.98       791



In [8]:
model.save_model("fall_xgb.json")
print("✅ Model saved")

✅ Model saved
